### Finding power series expansions along exceptional components
------------------------------------------------------------------------------------------
First we define the surface, for now only in affine form. This is the surface Wittenberg works with in his article

In [ ]:
A3<x,y,t>:=AffineSpace(Rationals(),3);
p:=3*(t-1)^3*(t+3);
q:=Evaluate(p,[x,y,-t]);
F:=y^2-x*(x-p)*(x-q);
E:=Surface(A3,F);

We check the discriminant of the elliptic curve. We need to do some hacky algebra to extract the polynomial we're interested in.

In [ ]:
Factorization(Discriminant(F-y^2,x));

Singular subscheme, and some of the points in the support

In [ ]:
Support(SingularSubscheme(E));

In [ ]:
Etilde:=DesingulariseSurfaceByBlowUp(E);

In [ ]:
function GetInitialTerm(f)
    if Valuation(f) eq 0 then
        return Coefficients(f)[1];
    else
        return Coefficients(f)[Valuation(f)];
    end if;
end function;

function ComputeResidue(f, g)
    vf:=Valuation(f);
    vg:=Valuation(g);
    
    f_0:=GetInitialTerm(f);
    g_0:=GetInitialTerm(g);
    
    return f_0^(vg)*g_0^(vf)*(-1)^(vf*vg);
end function;

### Computing residue at (-9, 0, 0) 

We unpack some of the desingularization data here. We pick one exceptional component L on a patch Y of the desingularization $\tilde{X}$ of X, together with the morphism $\pi: Y \to X$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[1],1);
A3<T,V,W>:=Ambient(Y);

We define a rational function field that we use to parametrize the exceptional component L with

In [ ]:
kU<U>:=FunctionField(Rationals(),1);

We write down a parametrization of L and check that it satisfies that parametrization. In particular, we see that "V" on Y vanishes on L (we'll see we can use it as a uniformizer at the generic point of L)

In [ ]:
Vu:=3/2*(U^2-1)/U;
Wu:=1/48*(U^2+1)/U;
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[0,Vu,Wu]) eq 0};

In [ ]:
kUT<Tu>:=PowerSeriesRing(kU);

In [ ]:
R<Wpol>:=PolynomialRing(kUT);

Another assumption: We set W to be the function in U that we need and we solve T as a power series in $k( U)[[V]]$ from the equation that we have. If V were not a valid choice as uniformizer, we'd probably fail to solve this equation at this point. Below we just compute all the roots and take the one close to the expression Tu that we already computed (because we know our series for T should specialize to Tu on L for V=0). It would be better to explicitly use newton iteration with initial approximation Tu.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Tu, Vu, Wpol]);
Fprime:=Derivative(F);
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series in $k(U)[[V]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Tu,Vu,Wu])+O(Tu^(10));

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $V,W,T$ (apologies for the mess-up in naming here: the two $T$s are not the same), and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Tu, Vu, Wu]);
Yu:=Evaluate(DefiningPolynomials(pi)[2], [Tu, Vu, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Tu, Vu, Wu]);

In [ ]:
Pu:=Evaluate(p, [0,0,Tu]);
Qu:=Evaluate(p, [0,0,-Tu]);
f1:=Xu-Pu;
f2:=Xu-Qu;

g1:=Evaluate(6*t*(t+1), [0,0,Tu]);
g2:=Evaluate(6*t*(t-1), [0,0,Tu]);

In [ ]:
res:=ComputeResidue(f1, g1)*ComputeResidue(f2, g2);
res;

Finally we observe that the residue is in fact a square, and therefore trivial.

In [ ]:
Factorization(Numerator(res));
Factorization(Denominator(res));
Factorization(Integers() ! Coefficients(Numerator(res))[1]);

### Computing residue at (0, 0, -3) 

We now pick the exceptional component L on the patch Y over $(0, -3, 0)$ of the desingularization $\tilde{E}$ of E, together with the morphism $\pi: Y \to X$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[2],1);
A3<X,V,W>:=Ambient(Y);

We define a rational function field that we use to parametrize the exceptional component L with

In [ ]:
kU<U>:=FunctionField(Rationals(),1);

We write down a parametrization of L and check that it satisfies that parametrization. In particular, we see that "V" on Y vanishes on L (we'll see we can use it as a uniformizer at the generic point of L)

In [ ]:
L;

Below we find a parametrization of $L$.

In [ ]:
Lc,phiL:=Conic(ProjectiveClosure(L));
piL:=Parametrization(Lc);
Q<x, v, w>:=FunctionField(L);
kU<U>:=FunctionField(Domain(piL));
Vu:=Pullback(piL, Pullback(Inverse(phiL), v));
Wu:=Pullback(piL, Pullback(Inverse(phiL), w));
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[0,Vu,Wu]) eq 0};
Vu;
Wu;

In [ ]:
kUX<Xu>:=PowerSeriesRing(kU);

In [ ]:
R<Wpol>:=PolynomialRing(kUX);

Another assumption: We set W to be the function in U that we need and we solve T as a power series in $k( U)[[V]]$ from the equation that we have. If V were not a valid choice as uniformizer, we'd probably fail to solve this equation at this point. Below we just compute all the roots and take the one close to the expression Tu that we already computed (because we know our series for T should specialize to Tu on L for V=0). It would be better to explicitly use newton iteration with initial approximation Tu.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Xu, Vu, Wpol]);
Fprime:=Derivative(F);
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series in $k(U)[[V]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Xu,Vu,Wu])+O(Xu^8);

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $V,W,T$ (apologies for the mess-up in naming here: the two $T$s are not the same), and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Xu, Vu, Wu]);
Yu:=Evaluate(DefiningPolynomials(pi)[2], [Xu, Vu, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Xu, Vu, Wu]);

In [ ]:
Pu:=Evaluate(p, [0,0,Tu]);
Qu:=Evaluate(p, [0,0,-Tu]);
f1:=Xu-Pu;
f2:=Xu-Qu;

g1:=Evaluate(6*t*(t+1), [0,0,Tu]);
g2:=Evaluate(6*t*(t-1), [0,0,Tu]);

In [ ]:
res:=ComputeResidue(f1, g1)*ComputeResidue(f2, g2);
res;

Here the residue is immediately seen to be a square.

## Computing residues at (0, 0, -1) 

### Patch 1

Here we pick the component L on the first affine patch Y over $(0,-3,0)$ of the desingularization $\tilde{X}$ of X, together with the morphism $\pi: Y \to X$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[3],1);

Looking at the equation of $L$ we note that it has no rational points and is geometrically reducible. This means that the residue field $k(L)$ will be of the form $k(u)$ with $k$ some finite extension of $\mathbb{Q}$; in our case $k=\mathbb{Q}(\sqrt{48})=\mathbb{Q}(\sqrt{3})$.

In [ ]:
A3<X,V,W>:=Ambient(Y);
L;
pi;

Note that $L:V^2-48=0, X=0$ has no rational points and is geometrically reducible. In particular, one can work out that the residue field is $\kappa(L)=k(W)$ where $k=\mathbb{Q}(a)$ and $a$ is a square root of $3$. Below we define the extension $k/\mathbb{Q}$ as well as a rational function field that we will use to parametrize the exceptional component L with.

In [ ]:
k<a>:=QuadraticField(48);
kU<U>:=FunctionField(k,1);
Wu:=U;

In [ ]:
kUX<Xu>:=PowerSeriesRing(kU);

In [ ]:
R<Vpol>:=PolynomialRing(kUX);

By definition we have an expression for $X$ and $W$ in $k(U)(Xu)$ as $W=U$ and $X=Xu$. It remains to find an expansion for $V$. From our observations above we note that when taking residues $\overline{V}=a=\sqrt{3}\in k(W)$, as we mod out by $V^2-48$, and so we have a first order approximation. Applying Newton iteration gives us enough terms to compute residues.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Xu, Vpol, Wu]);
Fprime:=Derivative(F);
Vu:=4*a;
for i in [1..4] do
    Temp:=Vu-Evaluate(F, Vu)/Evaluate(Fprime,Vu);
    Vu:=Temp;
end for;

We verify that the power series $Xu,Vu,Wu$ in $k(U)[[Xu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Xu,Vu,Wu])+O(Xu^10);

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $X,V,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Xu, Vu, Wu]);
Yu:=Evaluate(DefiningPolynomials(pi)[2], [Xu, Vu, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Xu, Vu, Wu]);

We now compute power series for our desired functions $X-p(T), X-q(T), 6T(T+1), 6T(T-1)$ in order to compute residues.

In [ ]:
Pu:=Evaluate(p, [0,0,Tu]);
Qu:=Evaluate(p, [0,0,-Tu]);
f1:=Xu-Pu;
f2:=Xu-Qu;

g1:=Evaluate(6*t*(t+1), [0,0,Tu]);
g2:=Evaluate(6*t*(t-1), [0,0,Tu]);

Finally, we compute the residue of the Brauer class $(f_1,g_1)+(f_2,g_2)$ with $f_1=X-p(T)$, $f_2=X-Q(T)$, $g_1=6T(T+1)$, and $g_2=6T(T-1)$ via the residue formula $\text{res}(f,g)=[(-1)^{v(f)v(g)}f_0^{v(g)}g_0^{v(f)}]$ where $f_0,g_0$ are the first nonzero coefficient in the power series expansion of $f_0,g_0$, and we take the equivalence class in $k(U)/k(U)^2$.

In [ ]:
res:=ComputeResidue(f1, g1)*ComputeResidue(f2, g2);
res;
Factorization(Integers() ! res);

Thus again we see that the residue is trivial.

### Patch 2

Here we pick the component L on the second affine patch Y over $(0,-3,0)$ of the desingularization $\tilde{X}$ of X, together with the morphism $\pi: Y \to X$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[3],2);

Looking at the equation of $L$ we note that it has no rational points and is geometrically reducible. This means that the residue field $k(L)$ will be of the form $k(u)$ with $k$ some finite extension of $\mathbb{Q}$; in our case $k=\mathbb{Q}(\sqrt{48})=\mathbb{Q}(\sqrt{3})$.

In [ ]:
A3<X,V,W>:=Ambient(Y);
L;
pi;

We define the extension $k/\mathbb{Q}$ as well as a rational function field that we will use to parametrize the exceptional component L with.

In [ ]:
k<a>:=QuadraticField(48);
kU<U>:=FunctionField(k,1);
Vu:=U;

In [ ]:
kUX<Xu>:=PowerSeriesRing(kU);

In [ ]:
R<Wpol>:=PolynomialRing(kUX);

By definition we have an expression for $X$ and $W$ in $k(U)(Xu)$ as $W=U$ and $X=Xu$. It remains to find an expansion for $V$. We note that when taking residues we have $\overline{V}=a=\sqrt{3}$, as we mod out by $V^2-48$, and so we have a first order approximation. Applying Newton iteration gives us enough terms to compute residues.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Xu, Vu, Wpol]);
Fprime:=Derivative(F);
Wu:=4*a;
for i in [1..4] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series $Xu,Vu,Wu$ in $k(U)[[Xu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Xu,Vu,Wu])+O(Xu^10);

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $X,V,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Xu, Vu, Wu]);
Yu:=Evaluate(DefiningPolynomials(pi)[2], [Xu, Vu, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Xu, Vu, Wu]);

We now compute power series for our desired functions $X-p(T), X-q(T), 6T(T+1), 6T(T-1)$ in order to compute residues.

In [ ]:
Pu:=Evaluate(p, [0,0,Tu]);
Qu:=Evaluate(p, [0,0,-Tu]);
f1:=Xu-Pu;
f2:=Xu-Qu;

g1:=Evaluate(6*t*(t+1), [0,0,Tu]);
g2:=Evaluate(6*t*(t-1), [0,0,Tu]);

Finally, we compute the residue of the Brauer class $(f_1,g_1)+(f_2,g_2)$ with $f_1=X-p(T)$, $f_2=X-Q(T)$, $g_1=6T(T+1)$, and $g_2=6T(T-1)$ via the residue formula $\text{res}(f,g)=[(-1)^{v(f)v(g)}f_0^{v(g)}g_0^{v(f)}]$ where $f_0,g_0$ are the first nonzero coefficient in the power series expansion of $f_0,g_0$, and we take the equivalence class in $k(U)/k(U)^2$.

In [ ]:
res:=ComputeResidue(f1, g1)*ComputeResidue(f2, g2);
res;

In [ ]:
Factorization(Integers() ! res);

Once again, the residue is trivial.

### Patch 3

Here we pick the component L on the third affine patch Y over $(0,-3,0)$ of the desingularization $\tilde{X}$ of X, together with the morphism $\pi: Y \to X$.

In [ ]:
L,Y,pi:=BlowUpDivisor(E,Etilde[3],3);

Looking at the equation of $L$ we note that it has no rational points and is geometrically reducible. This means that the residue field $k(L)$ will be of the form $k(u)$ with $k$ some finite extension of $\mathbb{Q}$; in our case $k=\mathbb{Q}(\sqrt{48})=\mathbb{Q}(\sqrt{3})$.

In [ ]:
A3<T,V,W>:=Ambient(Y);
L;
pi;

We write down a parametrization of our conic $L$. For the parametrization below we took the projective closure and projected away from the rational point $[0,0,1]$.

In [ ]:
kU<U>:=FunctionField(Rationals(), 1);
Vu:=(-6*U^2-1/8)/(-U^2+1/48)-6;
Wu:=-12*U/(-U^2+1/48);
assert forall{f: f in DefiningPolynomials(L) |Evaluate(f,[-1,Vu,Wu]) eq 0};

In [ ]:
kUT<Tu>:=PowerSeriesRing(kU);

In [ ]:
R<Wpol>:=PolynomialRing(kUT);

Now we compute an expansion for $W$ in $\mathbb{Q}(U)[[T]]$.

In [ ]:
F:=Evaluate(DefiningPolynomial(Y), [Tu-1, Vu, Wpol]);
Fprime:=Derivative(F);
for i in [1..2] do
    Temp:=Wu-Evaluate(F, Wu)/Evaluate(Fprime,Wu);
    Wu:=Temp;
end for;

We verify that the power series $Tu-1,Vu,Wu$ in $k(U)[[Tu]]$ that we have computed, indeed satisfy the defining polynomial for $Y$.

In [ ]:
Evaluate(DefiningPolynomial(Y),[Tu-1,Vu,Wu])+O(Tu^2);

In order to relate all this back to the original coordinates on X, we use the description of pi, that expresses $X,Y,T$ in terms of $X,V,W$, and since we have the latter as power series, we can substitute them to get the required results.

In [ ]:
Xu:=Evaluate(DefiningPolynomials(pi)[1], [Tu, Vu, Wu]);
Yu:=Evaluate(DefiningPolynomials(pi)[2], [Tu, Vu, Wu]);
Tu:=Evaluate(DefiningPolynomials(pi)[3], [Tu, Vu, Wu]);

We now compute power series for our desired functions $X-p(T), X-q(T), 6T(T+1), 6T(T-1)$ in order to compute residues.

In [ ]:
Pu:=Evaluate(p, [0,0,Tu]);
Qu:=Evaluate(p, [0,0,-Tu]);
f1:=Xu-Pu;
f2:=Xu-Qu;

g1:=Evaluate(6*t*(t+1), [0,0,Tu]);
g2:=Evaluate(6*t*(t-1), [0,0,Tu]);

Finally, we compute the residue of the Brauer class $(f_1,g_1)+(f_2,g_2)$ with $f_1=X-p(T)$, $f_2=X-Q(T)$, $g_1=6T(T+1)$, and $g_2=6T(T-1)$ via the residue formula $\text{res}(f,g)=[(-1)^{v(f)v(g)}f_0^{v(g)}g_0^{v(f)}]$ where $f_0,g_0$ are the first nonzero coefficient in the power series expansion of $f_0,g_0$, and we take the equivalence class in $k(U)/k(U)^2$.

In [ ]:
res:=ComputeResidue(f1, g1)*ComputeResidue(f2, g2);
res;

In [ ]:
Factorization(Numerator(res));
Factorization(Denominator(res));
Factorization(Integers() ! Coefficients(Numerator(res))[1]);

In [ ]:
Factorization(Integers() ! 81);

Once again, the residue is trivial.